## CNN - model Tuning - 1:

In [2]:
import zipfile

with zipfile.ZipFile('D:\Ai-Learning\Animal_Dataset.zip','r') as zip_ref:
    zip_ref.extractall('D:\Ai-Learning\Animal_Dataset')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### Data Augumentation

In [23]:
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, shear_range=0.2)
test_gen = ImageDataGenerator(rescale=1./255)

In [24]:
train = train_gen.flow_from_directory("D:\Ai-Learning\Animal_Dataset\dataset\Training", target_size=(224,224), batch_size=27, class_mode='categorical')
test = test_gen.flow_from_directory("D:\Ai-Learning\Animal_Dataset\dataset\Testing", target_size=(224,224), batch_size=27, class_mode='categorical')

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


##### Model Creation

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [10]:
tunnedmodel = Sequential()

In [ ]:
# add more layers to tune the model

In [11]:
tunnedmodel.add(Convolution2D(27, (3,3), activation='relu', input_shape=(224,224,3)))
tunnedmodel.add(MaxPooling2D(pool_size=(2,2)))

tunnedmodel.add(Convolution2D(35, (3,3)))
tunnedmodel.add(MaxPooling2D(pool_size=(2,2)))

tunnedmodel.add(Convolution2D(25, (3,3)))
tunnedmodel.add(MaxPooling2D(pool_size=(2,2)))

tunnedmodel.add(Flatten())

tunnedmodel.add(Dense(156, activation='relu'))
tunnedmodel.add(Dense(75, activation='relu'))
tunnedmodel.add(Dense(24, activation='relu'))
tunnedmodel.add(Dense(14, activation='relu'))

tunnedmodel.add(Dense(4, activation='softmax'))

In [12]:
tunnedmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
tunnedmodel.fit(train, validation_data=test, epochs=25, batch_size=27)

Epoch 1/25
46/46 [==============================] - 37s 795ms/step - loss: 1.3330 - accuracy: 0.3522 - val_loss: 1.2175 - val_accuracy: 0.4080
Epoch 2/25
46/46 [==============================] - 35s 766ms/step - loss: 1.1663 - accuracy: 0.4669 - val_loss: 0.9194 - val_accuracy: 0.6350
Epoch 3/25
46/46 [==============================] - 36s 785ms/step - loss: 0.9350 - accuracy: 0.6074 - val_loss: 0.8160 - val_accuracy: 0.6687
Epoch 4/25
46/46 [==============================] - 37s 803ms/step - loss: 0.8717 - accuracy: 0.6551 - val_loss: 0.7497 - val_accuracy: 0.7577
Epoch 5/25
46/46 [==============================] - 36s 770ms/step - loss: 0.7436 - accuracy: 0.7149 - val_loss: 0.6824 - val_accuracy: 0.7669
Epoch 6/25
46/46 [==============================] - 36s 778ms/step - loss: 0.6398 - accuracy: 0.7649 - val_loss: 0.5885 - val_accuracy: 0.7699
Epoch 7/25
46/46 [==============================] - 40s 858ms/step - loss: 0.5561 - accuracy: 0.7868 - val_loss: 0.3920 - val_accuracy: 0.8436

In [31]:
tunnedmodel.save('D:/Ai-Learning/tunnedmodel.h5')

In [17]:
# model - testing
import numpy as np
from tensorflow.keras.preprocessing import image

output = ['bear', 'crow', 'elephant', 'rat']
def func(str):
    img = image.load_img(str, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

img = func('DataSets/AnimalPredict/bear.jpg')
print(output[np.argmax(tunnedmodel.predict(img))])

img = func('DataSets/AnimalPredict/crow.jpg')
print(output[np.argmax(tunnedmodel.predict(img))])

img = func('DataSets/AnimalPredict/elephant.jpg')
print(output[np.argmax(tunnedmodel.predict(img))])

img = func('DataSets/AnimalPredict/rat.jpg')
print(output[np.argmax(tunnedmodel.predict(img))])

crow
crow
elephant
elephant


## Model Tunning - with
#### 1. Dropout
#### 2. Batch-Normalization
#### 3. Early Stopping

In [18]:
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
model2 = Sequential()

In [27]:
model2.add(Convolution2D(27, (3,3), activation='relu', input_shape=(224,224,3)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.2))

model2.add(Convolution2D(35, (3,3)))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.2))

model2.add(Convolution2D(25, (3,3)))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.2))

model2.add(Flatten())

model2.add(Dense(156, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(75, activation='relu'))
model2.add(Dense(24, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dense(14, activation='relu'))

model2.add(Dense(4, activation='softmax'))

In [28]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
mystop = EarlyStopping(monitor='accuracy',patience=5)
model2.fit(train, validation_data=test, epochs=100, batch_size=27 ,callbacks=mystop)

Epoch 1/100
46/46 [==============================] - 53s 1s/step - loss: 1.2781 - accuracy: 0.4548 - val_loss: 1.4190 - val_accuracy: 0.2239
Epoch 2/100
46/46 [==============================] - 55s 1s/step - loss: 0.9548 - accuracy: 0.6648 - val_loss: 1.3380 - val_accuracy: 0.3190
Epoch 3/100
46/46 [==============================] - 58s 1s/step - loss: 0.7611 - accuracy: 0.7456 - val_loss: 1.6123 - val_accuracy: 0.2270
Epoch 4/100
46/46 [==============================] - 55s 1s/step - loss: 0.6199 - accuracy: 0.7964 - val_loss: 1.6202 - val_accuracy: 0.2362
Epoch 5/100
46/46 [==============================] - 57s 1s/step - loss: 0.5350 - accuracy: 0.8086 - val_loss: 1.5443 - val_accuracy: 0.3006
Epoch 6/100
46/46 [==============================] - 60s 1s/step - loss: 0.4262 - accuracy: 0.8562 - val_loss: 1.3575 - val_accuracy: 0.3528
Epoch 7/100
46/46 [==============================] - 59s 1s/step - loss: 0.3527 - accuracy: 0.8772 - val_loss: 1.1959 - val_accuracy: 0.5123
Epoch 8/100
4

In [30]:
img = func('DataSets/AnimalPredict/bear.jpg')
print(output[np.argmax(model2.predict(img))])

img = func('DataSets/AnimalPredict/crow.jpg')
print(output[np.argmax(model2.predict(img))])

img = func('DataSets/AnimalPredict/elephant.jpg')
print(output[np.argmax(model2.predict(img))])

img = func('DataSets/AnimalPredict/rat.jpg')
print(output[np.argmax(model2.predict(img))])

bear
bear
bear
bear


In [32]:
model2.save('D:/Ai-Learning/tunnedmodel2.h5')